In [1]:
def get_gemini_api_key(config_path='gemini.config'):
    """
    Read the Gemini API key from the configuration file.
    
    Args:
        config_path (str): Path to the config file. Defaults to 'gemini.config' in the current directory.
    
    Returns:
        str: The API key as a string, or None if not found.
    """
    try:
        with open(config_path, 'r') as f:
            for line in f:
                if line.startswith('API_KEY='):
                    return line.split('=')[1].strip()
    except FileNotFoundError:
        print(f"Error: Config file not found at {config_path}")
        print("Please create a gemini.config file with your API key in the format:")
        print("API_KEY=your_api_key_here")
    except Exception as e:
        print(f"Error reading config file: {e}")
    
    return None